
Merge BIT image similarity and USE text similarity

Author: Christopher Pietsch

In [ ]:
import pandas as pd
import altair as alt
import numpy as np
import umap
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = pd.read_csv("export1305.csv").set_index('id')
data

,idlong,sammlung,idnr,titel,sachbegriff,actors,ort,datum,material,abmessung,beschreibung,datum_clean
id,,,,,,,,,,,,
968378,\tDE-MUS-815114/968378,ang,A I 232,Schafe im Erlenhain,Bild,"Heinrich von Zügel (22.10.1850 - 30.1.1941, Ma...",NaN,1875,Öl auf Leinwand,"Höhe x Breite: 45,5 x 72,5 cm","»Ich hatte das große Glück, niemals an einen B...",1875
959351,\tDE-MUS-815114/959351,ang,A III 730 (01),Einzug der Deutsch-Englischen Legion in Hannov...,Bild & Zyklus & Studie,"Ernst Hildebrand (8.3.1833 - 17.11.1924, Maler)",NaN,um 1900,Öl auf Leinwand,"Höhe x Breite: 65 x 39,5 cm;Rahmenmaß: 83 x 58...",Das mit Großbritannien seit 1714 durch Persona...,1900
962134,\tDE-MUS-815114/962134,ang,A I 1112,Campagna-Landschaft,Bild,"Arnold Böcklin (16.10.1827 - 16.1.1901, Maler)",NaN,um 1859,Öl auf Leinwand,Höhe x Breite: 88 x 105 cm;Rahmenmaß: 119 x 13...,Von 1850 bis 1857 lebte und arbeitete Arnold B...,1859
961042,\tDE-MUS-815114/961042,ang,A I 879,In der Laube,Bild,"Fritz Burger (16.7.1867 - 11.4.1927, Maler)",NaN,vor 1905,Öl auf Leinwand,"Höhe x Breite: 116 x 139,5 cm;Rahmenmaß: 140,5...",Fritz Burger schuf mit diesem Gemälde ein reiz...,1905
963307,\tDE-MUS-815114/963307,ang,A I 268,Der Verbrecher aus verlorener Ehre,Bild,Rudolf Friedrich August Henneberg (13.9.1825 -...,NaN,1860,Öl auf Leinwand,"Höhe x Breite: 197,5 x 135 cm","Allgemein wird vorausgesetzt, daß die authenti...",1860
...,...,...,...,...,...,...,...,...,...,...,...,...
567563,DE-MUS-018719/567563,mek,A (7 L 100) 11/1908,Tabakkasten zur ostfriesischen Winterküche,Tabakkasten zur ostfriesischen Winterküche,"James Simon (17.9.1851 - 23.5.1932, Sammler);C...","Gebrauchsort: Ostfriesland, Hannover",um 1800,"Holz, Messingknopf",Länge x Breite x Höhe: 19 x 15 x 14 cm,"Oval, auf dem Deckel Sechsstern und Messingkno...",1800
1486490,DE-MUS-018719/1486490,mek,D (44 B 1) 230/1966,Stempel der Kupferschmiedegesellen in Rostock,Stempel der Kupferschmiedegesellen in Rostock,NaN,"Gebrauchsort: Rostock, Mecklenburg",1851,"Messing, graviert; Holzgriff, gedrechselt","Höhe: 7,2 cm","Gedrechselter Holzgriff. Runde Messingplatte, ...",1851
1532768,DE-MUS-018719/1532768,mek,"D (62 C 25) 784/1989,b",Töchteralbum. 37. Band,Töchteralbum. 37. Band,Carl Flemming AG (Verleger);Thekla von Gumpert...,Herstellungsort: Glogau,1891,"Ganzleinen mit Deckel- und Rückenprägung, gekl...","Höhe x Breite x Tiefe: 23 x 16 x 3,5 cm",Hochformat. Töchter-Album. Unterhaltungen im h...,1891


In [ ]:
df_images = pd.read_csv("pca-bit.csv", index_col=0).set_index('id')
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(df_images) 
df_images_scaled = pd.DataFrame(scaled_values, index=df_images.index)
#df_images.iloc[:,:] = Normalizer(norm='l2').fit_transform(df_images)
df_images_scaled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1486397,0.341187,0.570036,0.516948,0.384615,0.731584,0.494372,0.306648,0.533106,0.563498,0.579646,0.417814,0.447848,0.382192,0.624746,0.339593,0.621802,0.708646,0.629764,0.692379,0.482235,0.463186,0.699846,0.568090,0.357213,0.186790,0.359196,0.350707,0.415219,0.903301,0.435259,0.295058,0.400715,0.253831,0.872703,0.457824,0.776966,0.367741,0.421124,0.420935,0.576680,...,0.483016,0.494754,0.450044,0.370944,0.430894,0.458111,0.648983,0.436842,0.604089,0.663801,0.441786,0.301560,0.446267,0.283554,0.593701,0.237624,0.273043,0.460621,0.472874,0.433613,0.524360,0.512482,0.505394,0.417937,0.381906,0.763841,0.349376,0.272281,0.577700,0.534527,0.724580,0.197793,0.570352,0.302459,0.302901,0.357035,0.381818,0.482085,0.384743,0.601027
1570538,0.446675,0.830590,0.362575,0.170049,0.493233,0.443290,0.492168,0.347050,0.264189,0.579119,0.361427,0.412716,0.542926,0.722660,0.662114,0.540283,0.700398,0.560209,0.679703,0.456948,0.359783,0.506780,0.366544,0.411436,0.484928,0.206947,0.287899,0.498436,0.431288,0.489318,0.542677,0.354414,0.409591,0.419828,0.341815,0.347378,0.266571,0.411982,0.514621,0.457708,...,0.527755,0.472962,0.457118,0.442478,0.686179,0.500891,0.519010,0.261404,0.465613,0.354346,0.490431,0.500000,0.539787,0.379017,0.406299,0.448845,0.428696,0.622912,0.581377,0.630252,0.652353,0.406021,0.331950,0.271749,0.394716,0.261246,0.416221,0.462796,0.340650,0.445013,0.514812,0.375212,0.438023,0.226230,0.278157,0.450832,0.598182,0.423453,0.421227,0.423801
525580,0.414742,0.847512,0.525114,0.323227,0.438785,0.070022,0.333449,0.573742,0.546278,0.468289,0.586975,0.952103,0.367011,0.259156,0.325470,0.378743,0.336427,0.404144,0.345648,0.515868,0.404371,0.401230,0.734493,0.699967,0.461703,0.422545,0.368464,0.447533,0.361500,0.438231,0.322731,0.505929,0.314945,0.337582,0.325586,0.445880,0.428278,0.559035,0.425033,0.403557,...,0.606462,0.483454,0.455349,0.435841,0.693496,0.460784,0.458886,0.528070,0.578067,0.545368,0.354864,0.482669,0.410993,0.500945,0.417323,0.444719,0.293043,0.537788,0.379757,0.453782,0.478943,0.472834,0.521162,0.557848,0.469976,0.512976,0.467914,0.454620,0.445127,0.484228,0.558046,0.353141,0.355109,0.450820,0.622867,0.534796,0.372727,0.420847,0.461857,0.441781
1442848,0.559293,0.542812,0.302160,0.698822,0.376279,0.207251,0.316858,0.425639,0.277444,0.456806,0.357836,0.305738,0.382447,0.555315,0.669719,0.473870,0.278097,0.184901,0.267738,0.252371,0.387797,0.662659,0.318573,0.244496,0.408335,0.563482,0.402992,0.539958,0.224255,0.821103,0.451033,0.324299,0.535785,0.416271,0.283711,0.535206,0.584855,0.625559,0.482399,0.167391,...,0.226181,0.260694,0.562334,0.502212,0.454472,0.453654,0.497790,0.537719,0.716543,0.406877,0.365231,0.275563,0.570960,0.743856,0.409449,0.416667,0.546957,0.388226,0.756275,0.605042,0.597027,0.294420,0.531120,0.521973,0.482786,0.576125,0.554367,0.459526,0.422300,0.779199,0.575661,0.505942,0.491625,0.462295,0.418089,0.456884,0.539091,0.525081,0.480100,0.512842
962622,0.769881,0.187437,0.808131,0.403909,0.483728,0.378030,0.460262,0.398309,0.523054,0.534450,0.500658,0.370532,0.113662,0.295524,0.682136,0.530076,0.418176,0.565280,0.744474,0.432798,0.460986,0.347966,0.525391,0.257148,0.432713,0.364520,0.348570,0.594684,0.375673,0.621958,0.446002,0.596462,0.439697,0.422940,0.716890,0.448876,0.340447,0.826298,0.439747,0.345850,...,0.553438,0.602098,0.394341,0.358407,0.609756,0.310160,0.660477,0.546491,0.449814,0.304680,0.234450,0.551127,0.431501,0.405482,0.388189,0.496700,0.669565,0.354813,0.496356,0.281513,0.481420,0.396476,0.528631,0.511211,0.539632,0.521626,0.277184,0.486509,0.490781,0.441603,0.559648,0.368421,0.396985,0.430328,0.433447,0.525719,0.650000,0.384365,0

In [ ]:
df_title = pd.read_csv("use.csv", index_col=0).set_index('id')
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(df_title) 
df_title_scaled = pd.DataFrame(scaled_values, index=df_title.index)
#df_title.iloc[:,:] = Normalizer(norm='l2').fit_transform(df_title)
df_title_scaled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
968378,0.535668,0.483663,0.763539,0.423278,0.767152,0.360539,0.449695,0.234610,0.232502,0.528075,0.646752,0.735349,0.585103,0.733666,0.381860,0.342081,0.464861,0.692109,0.367244,0.539606,0.369754,0.546254,0.449408,0.597222,0.653846,0.451408,0.414471,0.395798,0.281026,0.525630,0.418472,0.506791,0.375359,0.350964,0.796389,0.655602,0.145792,0.385860,0.625957,0.564774,...,0.502246,0.615540,0.821429,0.756569,0.540284,0.183753,0.755923,0.501319,0.486005,0.267621,0.204689,0.387195,0.628912,0.608643,0.546218,0.815974,0.622180,0.550798,0.592409,0.517550,0.394318,0.544915,0.632584,0.553177,0.623074,0.716470,0.186645,0.340126,0.565878,0.773800,0.413927,0.581724,0.327875,0.585165,0.321414,0.759225,0.535106,0.225882,0.723786,0.554824
959351,0.575443,0.468334,0.532795,0.307692,0.816008,0.170723,0.594893,0.279070,0.955350,0.402791,0.569767,0.279955,0.522130,0.511914,0.447027,0.310650,0.549666,0.643273,0.628427,0.386717,0.245297,0.688154,0.540778,0.486111,0.464585,0.390321,0.540778,0.648319,0.438702,0.580252,0.485901,0.405289,0.560986,0.569906,0.954284,0.489627,0.418412,0.517462,0.188587,0.762737,...,0.571429,0.157021,0.328725,0.673723,0.473539,0.432423,0.730350,0.449303,0.597237,0.459392,0.196573,0.309621,0.386735,0.368336,0.583270,0.905778,0.304511,0.468795,0.222360,0.500344,0.322348,0.560169,0.543071,0.293238,0.302647,0.371341,0.647034,0.327899,0.500670,0.368607,0.207412,0.439822,0.318736,0.352433,0.709120,0.807819,0.720177,0.558431,0.267238,0.574583
962134,0.460009,0.290843,0.394887,0.380991,0.588704,0.875328,0.774771,0.599521,0.191472,0.447257,0.569767,0.643897,0.354444,0.907379,0.321646,0.388160,0.621908,0.366106,0.611833,0.549291,0.405933,0.658434,0.615567,0.739464,0.355293,0.335978,0.362610,0.286975,0.546586,0.462185,0.624438,0.549678,0.585626,0.400574,0.480983,0.826556,0.346898,0.702300,0.736952,0.243450,...,0.539533,0.474302,0.413978,0.736861,0.494866,0.357219,0.731102,0.771956,0.484915,0.568157,0.515179,0.516938,0.582993,0.607431,0.314362,0.522635,0.596992,0.480044,0.431106,0.587749,0.396970,0.637288,0.736704,0.205573,0.677993,0.664919,0.461634,0.363097,0.753461,0.565303,0.303048,0.764487,0.510282,0.827316,0.452792,0.564852,0.316633,0.640000,0.389637,0.506005
961042,0.494163,0.247277,0.336697,0.338300,0.712058,0.484463,0.256860,0.371751,0.324216,0.309964,0.487570,0.392684,0.483267,0.507302,0.395960,0.254806,0.210051,0.739651,0.426046,0.563127,0.202243,0.835915,0.721489,0.600575,0.706398,0.551369,0.290255,0.394958,0.289702,0.429412,0.239477,0.657613,0.620945,0.485035,0.470611,0.495021,0.299881,0.824106,0.790884,0.769287,...,0.560647,0.456091,0.493472,0.258029,0.572670,0.327421,0.486273,0.716547,0.414758,0.269052,0.642020,0.466463,0.474150,0.657108,0.352941,0.489142,0.550752,0.683962,0.718647,0.398830,0.354924,0.168644,0.537453,0.496432,0.694192,0.301442,0.391539,0.477214,0.460473,0.629426,0.349074,0.486999,0.541127,0.520408,0.199679,0.751187,0.682004,0.576471,0.479396,0.500194
963307,0.496757,0.385639,0.535822,0.577930,0.858628,0.401722,0.751524,0.353283,0.602977,0.465433,0.423817,0.576708,0.581144,0.625288,0.416540,0.071102,0.389478,0.702781,0.740620,0.563819,0.189942,0.550440,0.621320,0.587165,0.592155,0.757239,0.329987,0.026050,0.228970,0.544538,0.506743,0.626519,0.197536,0.414924,0.469074,0.580913,0.185697,0.366695,0.469381,0.411208,...,0.356694,0.445569,0.632488,0.379562,0.648894,0.569706,0.702896,0.598568,0.553980,0.528801,0.286444,0.555556,0.468707,0.262924,0.521772,0.726169,0.704511,0.442671,0.334983,0.360633,0.385985,0.658898,0.473034,0.527693,0.732912,0.495850,0.584820,0.311967,0.778473,0.512982,0.162283,0.435736,0.403656,0.422684,0.328646,0.603946,0.521813,0.493333,0.49

In [ ]:
df_joined = df_title_scaled.join(df_images_scaled, lsuffix="i", rsuffix="t")
df_joined.dropna(inplace=True)
df_joined.iloc[:,:] = Normalizer(norm='l2').fit_transform(df_joined)
df_joined.describe()

,0i,1i,2i,3i,4i,5i,6i,7i,8i,9i,10i,11i,12i,13i,14i,15i,16i,17i,18i,19i,20i,21i,22i,23i,24i,25i,26i,27i,28i,29i,30i,31i,32i,33i,34i,35i,36i,37i,38i,39i,...,472t,473t,474t,475t,476t,477t,478t,479t,480t,481t,482t,483t,484t,485t,486t,487t,488t,489t,490t,491t,492t,493t,494t,495t,496t,497t,498t,499t,500t,501t,502t,503t,504t,505t,506t,507t,508t,509t,510t,511t
count,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,...,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000,5140.000000
mean,0.033084,0.028055,0.032500,0.028959,0.037054,0.030978,0.031683,0.029707,0.031263,0.025797,0.032018,0.033705,0.031617,0.036436,0.028464,0.018815,0.031504,0.037894,0.032749,0.026817,0.027794,0.032986,0.037748,0.030374,0.037336,0.032254,0.031162,0.028746,0.029696,0.030817,0.028549,0.041414,0.030522,0.031358,0.031101,0.032683,0.027638,0.030732,0.029567,0.026373,...,0.030319,0.028933,0.028862,0.027031,0.031607,0.027282,0.031317,0.029599,0.029873,0.028280,0.026228,0.029409,0.031790,0.029569,0.027357,0.031007,0.030261,0.027537,0.029530,0.028106,0.029860,0.027676,0.027142,0.030165,0.027123,0.031971,0.029968,0.029667,0.027686,0.030356,0.030336,0.025561,0.029509,0.026912,0.031169,0.028192,0.028333,0.028984,0.027931,0.027367
std,0.010048,0.009800,0.009943,0.008700,0.012421,0.009244,0.010676,0.009506,0.010323,0.006095,0.010290,0.009660,0.009860,0.009414,0.009614,0.007279,0.010661,0.008297,0.010301,0.008169,0.009495,0.008345,0.009515,0.009325,0.009009,0.010133,0.009914,0.010149,0.009886,0.010308,0.008485,0.007598,0.009403,0.009260,0.010131,0.009678,0.010662,0.010632,0.010015,0.010430,...,0.007528,0.007293,0.008002,0.006665,0.007332,0.008016,0.007952,0.007871,0.008304,0.008524,0.007099,0.007695,0.007276,0.008365,0.006941,0.007289,0.007669,0.006968,0.007112,0.007345,0.007235,0.006419,0.007201,0.007738,0.006937,0.007503,0.007682,0.007019,0.007498,0.007274,0.006803,0.007242,0.007092,0.006913,0.007204,0.006351,0.007666,0.005459,0.006980,0.007175
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.025979,0.021053,0.025195,0.022694,0.026767,0.024715,0.024073,0.022752,0.023606,0.021960,0.024734,0.026804,0.024596,0.030319,0.021840,0.014141,0.023847,0.032453,0.026091,0.020919,0.020974,0.027203,0.032237,0.023933,0.031546,0.025199,0.024027,0.021771,0.022678,0.023472,0.022789,0.037157,0.023872,0.024791,0.023943,0.026028,0.019569,0.022851,0.022240,0.018450,...,0.025410,0.024262,0.023570,0.022664,0.026840,0.021820,0.026176,0.024576,0.024534,0.022625,0.021507,0.024476,0.026995,0.024

In [ ]:
def runUmapOnDf(df):
  umap_op = umap.UMAP(n_components=2, n_neighbors=100,min_dist=0.2)
  data_umap = umap_op.fit_transform(df.values)
  data_umap_df = pd.DataFrame(data=data_umap, columns=['x', 'y'])
  data_umap_df['id'] = df.index
  return data_umap_df.set_index('id')

In [ ]:
df_joined_umap = runUmapOnDf(df_joined).join(data['sammlung'])
alt.data_transformers.disable_max_rows()
alt.Chart(df_joined_umap).encode(
      x='x',
      y='y',
      color='sammlung'
  ).mark_circle(size=60).properties(
      width=600, height=600
  ).interactive()

alt.Chart(...)

In [ ]:
df_joined_umap3 = runUmapOnDf(df_images).join(data['sammlung'])
alt.data_transformers.disable_max_rows()
alt.Chart(df_joined_umap3).encode(
      x='x',
      y='y',
      color='sammlung'
  ).mark_circle(size=60).properties(
      width=600, height=600
  ).interactive()

alt.Chart(...)

In [ ]:
df_joined_umap2 = runUmapOnDf(df_title).join(data['sammlung'])
alt.data_transformers.disable_max_rows()
alt.Chart(df_joined_umap2).encode(
      x='x',
      y='y',
      color='sammlung'
  ).mark_circle(size=60).properties(
      width=600, height=600
  ).interactive()

alt.Chart(...)

In [ ]:
df_joined_umap2

,x,y,sammlung
id,,,
1486397,3.327157,6.909046,mek
1570538,3.721031,12.012905,mek
525580,6.183940,10.559856,mek
1442848,-0.287874,10.157651,mek
962622,0.262986,15.192502,ang
...,...,...,...
984816,5.278814,9.231367,mek
1479834,1.868817,5.283248,mek
961858,-6.149833,11.688212,ang


In [ ]:
df_joined_umap.to_csv("pca-use-2d.csv")